# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import json

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()


In [3]:
def get_files(filepath):
    all_files = []

    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
songs = get_files('data/song_data')

In [5]:
filepath =songs[0]
print(filepath)



/home/workspace/data/song_data/A/A/C/TRAACER128F4290F96.json


In [6]:
df = pd.read_json(filepath, typ='series')
df

    

num_songs                                         1
artist_id                        ARMAC4T1187FB3FA4C
artist_latitude                             40.8262
artist_longitude                             -74.48
artist_location                   Morris Plains, NJ
artist_name               The Dillinger Escape Plan
song_id                          SOBBUGU12A8C13E95D
title               Setting Fire to Sleeping Giants
duration                                    207.778
year                                           2004
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[["song_id","title","artist_id","year","duration"]]
song_values = song_data.values

first_song = song_values[:]
song_data = first_song.tolist()


assert isinstance(song_data, list)
song_data


    

['SOBBUGU12A8C13E95D',
 'Setting Fire to Sleeping Giants',
 'ARMAC4T1187FB3FA4C',
 2004,
 207.77751]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]]
artist_values = artist_data.values

first_artist = artist_values[:]
artist_data = first_artist.tolist()


assert isinstance(artist_data, list)
artist_data

['ARMAC4T1187FB3FA4C',
 'The Dillinger Escape Plan',
 'Morris Plains, NJ',
 40.82624,
 -74.47995]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

In [12]:
filepath = log_files[0]
print(filepath)

/home/workspace/data/log_data/2018/11/2018-11-12-events.json


In [13]:
df = pd.read_json(filepath, lines=True)
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Celeste,F,0,Williams,NaN,free,"Klamath Falls, OR",GET,Home,1.541078e+12,438,None,200,1541990217796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
1,Pavement,Logged In,Sylvie,F,0,Cruz,99.16036,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,345,Mercy:The Laundromat,200,1541990258796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
2,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Celeste,F,1,Williams,277.15873,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1541990264796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
3,Gary Allan,Logged In,Celeste,F,2,Williams,211.22567,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Nothing On But The Radio,200,1541990541796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
4,None,Logged In,Jacqueline,F,0,Lynch,NaN,paid,"Atlanta-Sandy Springs-Roswell, GA",GET,Home,1.540224e+12,389,None,200,1541990714796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29
5,Charttraxx Karaoke,Logged In,Celeste,F,3,Williams,225.17506,free,"Klamath Falls, OR",PUT,NextSong,1.541078e+12,438,Fireflies,200,1541990752796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",53
6,The Libertines,Logged In,Jacqueline,F,1,Lynch,179.53914,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,The Good Old Days,200,1541990842796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29
7,Huey Lewis And The News,Logged In,Jacqueline,F,2,Lynch,245.52444,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,Hip To Be Square,200,1541991021796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29
8,JOSEF LOCKE & ORCHESTRA,Logged In,Jacqueline,F,3,Lynch,166.26893,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,How Can You Buy Killarney (1992 Digital Remaster),200,1541991266796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29
9,Train,Logged In,Jacqueline,F,4,Lynch,216.76363,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,NextSong,1.540224e+12,389,Hey_ Soul Sister,200,1541991432796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df['page'] == 'NextSong']

In [15]:
timest = pd.to_datetime(df['ts'], unit='ms')

In [16]:
time_data = [timest, timest.dt.hour, timest.dt.day, timest.dt.weekofyear, timest.dt.month, timest.dt.year,timest.dt.weekday]
column_labels = ['ts', 'hour', 'day', 'week of year', 'month', 'year', 'weekday']

assert isinstance(time_data, list), 'time_data should be a list'
assert isinstance(column_labels, list), 'column_labels should be a list'



In [17]:
dictionary = dict(zip(column_labels, time_data))
time_df = pd.DataFrame.from_dict(dictionary)
assert isinstance(time_df, pd.DataFrame), 'time_df should be a dataframe'

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
user_df = user_df.drop_duplicates().sort_values(['userId'])

for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
import datetime

for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()

    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    #String myTimestamp = Instant.now().toString()
    # insert songplay record 

    #new_timestamp = datetime.datetime.fromtimestamp(row.ts / 1e3)
    new_timestamp = pd.to_datetime(row.ts, unit='ms')
    
    songplay_data = (new_timestamp, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.